In [ ]:
import numpy as np
import scipy.special as sc
import matplotlib.pyplot as plt
from scipy.io import loadmat
from ipywidgets import interact, interactive, widgets

from scipy.constants import h, c, k, e, m_e, epsilon_0,physical_constants

k_ev = physical_constants['Boltzmann constant in eV/K'][0]
sigma_sb = physical_constants['Stefan-Boltzmann constant'][0]
L = 2.44e-8

In [ ]:
def rd_current(phi, T):
    """
    Thermionic emission current density in A/m**2
    """
    A = 4 * np.pi * m_e * k**2 * e / h**3
    
    return A * T**2 * np.exp(-phi / (k_ev * T))

def Vretard(J,phi_c, T):
    """
    Thermionic emission current density in A/m**2
    """
    A = 4 * np.pi * m_e * k**2 * e / h**3
    
    return -np.log(J/(A * T**2)) *  (k_ev * T)-phi_c


def x0_fun(J,T):
    x0 = (epsilon_0**2 * k**3/2/np.pi/m_e/e**2)**(1/4)*T**(3/4)/J**(1/2)
    return x0

def xiFromJ(J,x,xm,T):
    x0 = (epsilon_0**2 * k**3/2/np.pi/m_e/e**2)**(1/4)*T**(3/4)/J**(1/2)
    xi = (x-xm)/x0
    
    return xi

def JFromxi(xi,x,xm,T):
    x0 = (x-xm)/xi
    J = ((epsilon_0**2 * k**3/2/np.pi/m_e/e**2)**(1/4)*T**(3/4)/x0)**2
    
    return J

def calculate_currfraction(T, V_gap, drefprob, E_arr=None):
    if E_arr is None:
        if V_gap > 0:
            frac= (1-drefprob) * (1 + drefprob * e * V_gap / k / T + drefprob * (1 - drefprob) * (e * V_gap / k / T) ** 2 * np.exp((1 - drefprob) * e * V_gap / k / T ) * sc.expi(-(1 - drefprob) * e * V_gap / k / T))
        else:
            frac = (1-drefprob)   
    else:
        if len(drefprob) != len(E_arr):
            raise RuntimeError("energy array (E_arr) and diffusive reflectivity (drefprob) must be in the same size")
        frac = np.trapz(np.exp(-E_arr / k / T) * (1 - drefprob) / (1 - drefprob * e * V_gap / (e * V_gap + E_arr)) * E_arr / (k * T) ** 2, E_arr)
    
    return frac

In [ ]:
def calculate_J(phi_e, phi_c, T_e, d, gammas, xim, xip, l_sat = False):
    J_sat = rd_current(phi_e,T_e)
    xi_e = xiFromJ(J_sat,d,0,T_e)
    gamma_e = np.interp(xi_e,xip,gammas)
    V_sat = phi_e-phi_c-gamma_e*k_ev*T_e
#     print("gamma_e is:", gamma_e)
#     print("V_sat is:",V_sat)
    
    if V_sat > phi_e-phi_c or l_sat:
        J_res = J_sat
        print("No reflection. T_e:",round(T_e - 273.15),"C, saturation current: ", J_res)
        return J_res
    
    # first guess of the critical point:
    J_c_0 = JFromxi(max(np.abs(xim)),0,d,T_e)
    gamma_E = np.log(J_sat/J_c_0)
    V_c_0 = -(gamma_E*k*T_e/e-phi_e+phi_c)

    # critical point:
    J_arr = np.exp(np.linspace(-6,3,100000))*J_c_0
    gamma_e_arr = np.log(J_sat/J_arr)
    excludeind = np.where(gamma_e_arr<0)
    np.put(gamma_e_arr,excludeind,0)
    xi_e_arr = np.interp(gamma_e_arr, gammas,xim)
    xi_c_arr = d/x0_fun(J_arr, T_e) + xi_e_arr
    np.put(xi_c_arr,excludeind,-1)
    xi_c = min(xi_c_arr[xi_c_arr>=0])
    J_r = J_arr[np.where(xi_c_arr==xi_c)[0][0]]
    V_r = Vretard(J_r, phi_c, T_e)
#     print("V_cr", V_r)
    
    if V_r < phi_e-phi_c:
        J_res = rd_current(phi_c + phi_e - phi_c,T_e)
        print("No reflection. T_e:",round(T_e - 273.15),"C, retard current: ", J_res)
        return J_res
    
    # space charge limited iv:
    n_scl = 1000000
    J_scl = np.linspace(J_sat,J_r,n_scl)
    gamma_e_scl = np.log(J_sat/J_scl)
    xi_e_scl = np.interp(gamma_e_scl, gammas,xim)
    xi_c_scl = d/x0_fun(J_scl, T_e) + xi_e_scl
    gamma_c_scl = np.interp(xi_c_scl,xip,gammas)
    V_scl = phi_e-phi_c+(gamma_e_scl-gamma_c_scl)*k*T_e/e
    J_res = np.interp(phi_e-phi_c, V_scl, J_scl)
    print("No reflection. T_e:",round(T_e - 273.15),"C, SCL current: ", J_res)
    return J_res

def calculate_J_sr(phi_e, phi_c, T_e, d, gammas, xim, xip, srefprob, l_sat = False):
    J_sat = rd_current(phi_e,T_e)
    xi_e = xiFromJ(J_sat,d,0,T_e)
    gamma_e = np.interp(xi_e,xip,gammas)
    V_sat = phi_e-phi_c-gamma_e*k_ev*T_e
#     print("V_sat is:",V_sat)
    
    if V_sat > phi_e-phi_c or l_sat:
        J_res = J_sat * (1 - srefprob)
        print("Specular reflection. T_e:",round(T_e - 273.15),"C, saturation current: ", J_res)
        return J_res
    
    # first guess of the critical point:
    J_c_0 = JFromxi(max(np.abs(xim)),0,d,T_e)
    gamma_E = np.log(J_sat/J_c_0)
    V_c_0 = -(gamma_E*k*T_e/e-phi_e+phi_c)

    # critical point:
    J_arr = np.exp(np.linspace(-6,3,100000))*J_c_0
    gamma_e_arr = np.log(J_sat/J_arr)
    excludeind = np.where(gamma_e_arr<0)
    np.put(gamma_e_arr,excludeind,0)
    xi_e_arr = np.interp(gamma_e_arr, gammas,xim)
    xi_c_arr = d/x0_fun(J_arr, T_e) + xi_e_arr
    np.put(xi_c_arr,excludeind,-1)
    xi_c = min(xi_c_arr[xi_c_arr>=0])
    J_r = J_arr[np.where(xi_c_arr==xi_c)[0][0]]
    V_r = Vretard(J_r, phi_c, T_e)
#     print("V_cr", V_r)
    
    if V_r < phi_e-phi_c:
        J_res = rd_current(phi_c + phi_e - phi_c,T_e) * (1 - srefprob)
        print("Specular reflection. T_e:",round(T_e - 273.15),"C, retard current: ", J_res)
        return J_res
    
    # space charge limited iv:
    n_scl = 1000000
    J_scl = np.linspace(J_sat,J_r,n_scl)
    gamma_e_scl = np.log(J_sat/J_scl)
    xi_e_scl = np.interp(gamma_e_scl, gammas,xim)
    xi_c_scl = d/x0_fun(J_scl, T_e) + xi_e_scl
    gamma_c_scl = np.interp(xi_c_scl,xip,gammas)
    V_scl = phi_e-phi_c+(gamma_e_scl-gamma_c_scl)*k*T_e/e
    J_res = np.interp(phi_e-phi_c, V_scl, J_scl) * (1 - srefprob)
    print("Specular reflection. T_e:",round(T_e - 273.15),"C, SCL current: ", J_res)
    return J_res

def calculate_J_dr(phi_e, phi_c, T_e, d, gammas, xim, xip, drefprob, l_sat = False):
    J_sat = rd_current(phi_e,T_e)
    xi_e = xiFromJ(J_sat,d,0,T_e)
    gamma_e = np.interp(xi_e,xip,gammas)
    V_sat = phi_e-phi_c-gamma_e*k_ev*T_e
#     print("V_sat is:",V_sat)
    if V_sat > phi_e-phi_c or l_sat:
        J_res = J_sat * calculate_currfraction(T_e, phi_e-phi_c, drefprob)
        print("Diffuse reflection. T_e:",round(T_e - 273.15),"C, saturation current: ", J_res)
        return J_res
    
    # first guess of the critical point:
    J_c_0 = JFromxi(max(np.abs(xim)),0,d,T_e)
    gamma_E = np.log(J_sat/J_c_0)
    V_c_0 = -(gamma_E*k*T_e/e-phi_e+phi_c)

    # critical point:
    J_arr = np.exp(np.linspace(-6,3,100000))*J_c_0
    gamma_e_arr = np.log(J_sat/J_arr)
    excludeind = np.where(gamma_e_arr<0)
    np.put(gamma_e_arr,excludeind,0)
    xi_e_arr = np.interp(gamma_e_arr, gammas,xim)
    xi_c_arr = d/x0_fun(J_arr, T_e) + xi_e_arr
    np.put(xi_c_arr,excludeind,-1)
    xi_c = min(xi_c_arr[xi_c_arr>=0])
    J_r = J_arr[np.where(xi_c_arr==xi_c)[0][0]]
    V_r = Vretard(J_r, phi_c, T_e)
#     print("V_cr", V_r)
    
    if V_r < phi_e-phi_c:
        J_res = rd_current(phi_c + phi_e - phi_c,T_e) * calculate_currfraction(T_e, phi_e - phi_c, drefprob)
        print("Diffuse reflection. T_e:",round(T_e - 273.15),"C, retard current: ", J_res)
        return J_res
    
    # space charge limited iv:
    n_scl = 1000000
    J_scl = np.linspace(J_sat,J_r,n_scl)
    gamma_e_scl = np.log(J_sat/J_scl)
    xi_e_scl = np.interp(gamma_e_scl, gammas,xim)
    xi_c_scl = d/x0_fun(J_scl, T_e) + xi_e_scl
    gamma_c_scl = np.interp(xi_c_scl,xip,gammas)
    V_scl = phi_e-phi_c+(gamma_e_scl-gamma_c_scl)*k_ev*T_e
    J_scl0 = np.interp(phi_e-phi_c, V_scl, J_scl)
    V_a0 = (np.log(J_sat/J_scl0))*k_ev*T_e
    J_res = J_scl0 * calculate_currfraction(T_e, V_a0, drefprob)
    print("Diffuse reflection. T_e:",round(T_e - 273.15),"C, SCL current: ", J_res)
    return J_res
def calculate_efficiency(A_em,J_em, J_coll, phi_em, T_em, rho_em,
               rho_ew, rho_cw, phi_coll, T_coll,
               emiss_eff, T_env=293.15,
               rho_load=0, single_cycle=True):
    """
    Calculate the TEC efficieny.
    All power terms should be calculated to give W/cm**2
    Based on S. Meir et al. J. Renewable Sustainable Energy 2013.
    Args:
        rho_ew: Effective emitter wiring resistivity (Ohms*cm)
        J_em: Emitter current density (A/cm**2)
        J_coll: Collector current density (A/cm**2)
        phi_em: Emitter work function (eV)
        T_em: Emitter temperature (K)
        rho_cw: Effective collector wiring resistivity (Ohms*cm)
        phi_coll: Collector work function (eV)
        T_coll: Collector temperature (K)
        emiss_eff: Emissivity (none)
        T_env: Ambient temperature (K)
        rho_load: Effective load resistivity (Ohms*cm)
                  if set < 0 then perfect matching is assumed -> V_load = phi_em - phi_col.
        single_cycle: True if single cycle, False if combined cycle
        **kwargs: Catch unused arguments

    Returns:
        Efficiency (none)
    """
    
    # Turning off analytic backward current. Is a small effect but creates misleading results in low forward current
    #  cases encountered frequently in optimization

    # Modify measured J_ec (emitter to collector current) to remove analytical collector produced current
    J_load = (J_em - J_coll)
    
    # P_ew
    P_ew = 0.5 * (L / (rho_em * A_em) * (T_em - T_env) ** 2 - (J_load) ** 2 * A_em * rho_ew)
    
    # P_cw
    P_cw = -0.5 * (L / (rho_cw * A_em) * (T_coll - T_env) ** 2 - (J_load) ** 2 * A_em * rho_cw)

    # P_r
    P_r = emiss_eff * sigma_sb * (T_em ** 4 - T_coll ** 4) * 1e-4

    # P_ec (electron cooling power)
    P_ec = J_em * (phi_em + 2 * k_ev * T_em) - J_coll * (phi_em + 2 * k_ev * T_coll)
    
    # P_eh (electron heating of the collector)
    P_eh = J_em * (phi_coll + 2 * k_ev * T_em) - J_coll * (phi_coll + 2 * k_ev * T_coll)

    # P_load
    V_lead = J_load * A_em * (rho_cw + rho_ew)
    # TODO: Check R_total for adjustment in rho
    if rho_load > 0:
        R_total = rho_load
        V_load = R_total * A_em * J_load - V_lead
    else:
        V_load = (phi_em - phi_coll) - V_lead
    P_load = J_load * V_load

    # P_gate
    if single_cycle:

        eta = (P_load) / (P_ec + P_r + P_ew )

        efficiency_data = {}
        efficiency_data['P_ew'] = P_ew
        efficiency_data['P_r'] = P_r
        efficiency_data['P_ec'] = P_ec
        efficiency_data['P_load'] = P_load
        efficiency_data['eta'] = eta
    else:
        eta_s = 0.32
        Q_out = P_eh + P_r + P_cw
        eta = (P_load + eta_s * Q_out) / (P_ec + P_r + P_ew )

        efficiency_data = {}
        efficiency_data['P_ew'] = P_ew
        efficiency_data['P_r'] = P_r
        efficiency_data['P_ec'] = P_ec
        efficiency_data['P_load'] = P_load
        efficiency_data['P_s'] = eta_s * Q_out
        efficiency_data['eta'] = eta
        

    return efficiency_data

In [ ]:
drefprob = 0.5
srefprob = 0.5
data = np.genfromtxt('./data_table.csv',delimiter=',')
gammas = data[:,0]
xim = data[:,1]
xip = data[:,2]
data = np.genfromtxt('./data_table_dr'+str(drefprob)+'.csv',delimiter=',')
gammas_dr = data[:,0]
xim_dr = data[:,1]
xip_dr = data[:,2]
data = np.genfromtxt('./data_table_sr'+str(srefprob)+'.csv',delimiter=',')
gammas_sr = data[:,0]
xim_sr = data[:,1]
xip_sr = data[:,2]

In [ ]:
phi_e = 2.174 #eV 2.139#
phi_c = 0.381 #eV 0.877#
T_es = np.linspace(1200,1700,50) + 273.15 #K
T_esnb = np.linspace(1000,1700,50) + 273.15 #K
T_es_no = np.linspace(1000,5000,41) + 273.15 #K
T_es_dr = np.linspace(1000,5000,41) + 273.15 #K
T_es_sr = np.linspace(1000,5000,41) + 273.15 #K
d = 10e-6 #m
if_sat = False

A_em = 1
T_coll = 50 + 273.15
emiss_eff = 0.2
T_env = 293.15
rho_em = 1e-3
rho_cw = 1e-3
rho_ew = 1e-3
efficiencies = []
for T_e in T_es:
    J_coll = rd_current(phi_c, T_coll)*1e-4
    J_em = rd_current(phi_e, T_e)*1e-4

    efficiencies.append(calculate_efficiency(A_em, J_em, J_coll, phi_e, T_e, rho_em,
                   rho_ew, rho_cw, phi_c, T_coll,
                   emiss_eff, T_env,single_cycle=1))
efficienciesnb = []
for T_e in T_esnb:
    J_coll = 0#rd_current(phi_c, T_coll)*1e-4
    J_em = rd_current(phi_e, T_e)*1e-4

    efficienciesnb.append(calculate_efficiency(A_em, J_em, J_coll, phi_e, T_e, rho_em,
                   rho_ew, rho_cw, phi_c, T_coll,
                   emiss_eff, T_env,single_cycle=1))
efficiencies_no = []
for T_e in T_es_no:
    J_coll = 0#rd_current(phi_c, T_coll)*1e-4
    J_em = calculate_J(phi_e, phi_c, T_e, d, gammas, xim, xip, l_sat = if_sat)*1e-4

    efficiencies_no.append(calculate_efficiency(A_em, J_em, J_coll, phi_e, T_e, rho_em,
                   rho_ew, rho_cw, phi_c, T_coll,
                   emiss_eff, T_env,single_cycle=1))
efficiencies_dr = []
for T_e in T_es_dr:
    J_coll = 0#rd_current(phi_c, T_coll)*1e-4
    J_em = calculate_J_dr(phi_e, phi_c, T_e, d, gammas_dr, xim_dr, xip_dr, drefprob, l_sat = if_sat)*1e-4#rd_current(phi_e, T_e)

    efficiencies_dr.append(calculate_efficiency(A_em, J_em, J_coll, phi_e, T_e, rho_em,
                   rho_ew, rho_cw, phi_c, T_coll,
                   emiss_eff, T_env,single_cycle=1))
efficiencies_sr = []
for T_e in T_es_sr:
    J_coll = 0#rd_current(phi_c, T_coll)*1e-4
    J_em = calculate_J_sr(phi_e, phi_c, T_e, d, gammas_sr, xim_sr, xip_sr, srefprob, l_sat = if_sat)*1e-4#rd_current(phi_e, T_e)

    efficiencies_sr.append(calculate_efficiency(A_em, J_em, J_coll, phi_e, T_e, rho_em,
                   rho_ew, rho_cw, phi_c, T_coll,
                   emiss_eff, T_env,single_cycle=1))

## _Plot efficiency versus barrier thickness_

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

# eta = np.zeros_like(voltages)
# for i, efficiency in enumerate(efficiencies00):
#     eta = np.zeros_like(voltages)
#     for j, eff in enumerate(efficiency):
#         eta[j] = eff['eta']
#     ax.plot(voltages, eta, "<", lw=2, label=r"drefprob ="+str(drefprobs_const[i]))
eta = np.zeros_like(T_es)
for i, efficiency in enumerate(efficiencies):
    eta[i] = efficiency['eta']
ax.plot(T_es-273.15, eta, label = 'single cycle, Voesch et al.')
etanb = np.zeros_like(T_esnb)
for i, efficiency in enumerate(efficienciesnb):
    etanb[i] = efficiency['eta']
ax.plot(T_esnb-273.15, etanb, label = 'single cycle, Voesch et al. no back emission')
eta_no = np.zeros_like(T_es_no)
for i, efficiency in enumerate(efficiencies_no):
    eta_no[i] = efficiency['eta']
ax.plot(T_es_no-273.15, eta_no, label = 'single cycle, 10um, no reflection')#diffuse reflection 0.5

eta_dr = np.zeros_like(T_es_dr)
for i, efficiency in enumerate(efficiencies_dr):
    eta_dr[i] = efficiency['eta']
ax.plot(T_es_dr-273.15, eta_dr, label = 'single cycle, 10um, diffusive reflection 0.5')#diffuse reflection 0.5

eta_sr = np.zeros_like(T_es_sr)
for i, efficiency in enumerate(efficiencies_sr):
    eta_sr[i] = efficiency['eta']
ax.plot(T_es_sr-273.15, eta_sr, label = 'single cycle, 10um, specular reflection 0.5')#diffuse reflection 0.5

ax.legend(loc="upper right")

ax.set_xlim([1000, 5000])
ax.set_ylim([0.0, 0.65])#0.325

ax.set_xlabel("T_em (C)")
ax.set_ylabel("efficiency, $\\eta$")
# plt.savefig('benchmark_Voesch_fig8a_10um_reflection05.png')

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

# eta = np.zeros_like(voltages)
# for i, efficiency in enumerate(efficiencies00):
#     eta = np.zeros_like(voltages)
#     for j, eff in enumerate(efficiency):
#         eta[j] = eff['eta']
#     ax.plot(voltages, eta, "<", lw=2, label=r"drefprob ="+str(drefprobs_const[i]))
# eta = np.zeros_like(T_es)

for i, efficiency in enumerate(efficiencies):
    eta[i] = efficiency['P_load']
ax.semilogy(T_es-273.15, eta, label = 'single cycle, Voesch et al.')
for i, efficiency in enumerate(efficienciesnb):
    eta[i] = efficiency['P_load']
ax.semilogy(T_esnb-273.15, eta, label = 'single cycle, Voesch et al., no back emission')
eta_no = np.zeros_like(T_es_no)
for i, efficiency in enumerate(efficiencies_no):
    eta_no[i] = efficiency['P_load']
ax.semilogy(T_es_no-273.15, eta_no, label = 'single cycle, 10um, no reflection')
eta_dr = np.zeros_like(T_es_dr)
for i, efficiency in enumerate(efficiencies_dr):
    eta_dr[i] = efficiency['P_load']
ax.plot(T_es_dr-273.15, eta_dr, label = 'single cycle, 10um, diffusive reflection 0.5')
eta_sr = np.zeros_like(T_es_sr)
for i, efficiency in enumerate(efficiencies_sr):
    eta_sr[i] = efficiency['P_load']
ax.semilogy(T_es_sr-273.15, eta_sr, label = 'single cycle, 10um, specular reflection 0.5')
ax.legend(loc="upper left")

ax.set_xlim([1000, 5000])
ax.set_ylim([1e-1, 2.5e3])#1e1

ax.set_xlabel("T_em (C)")
ax.set_ylabel("P, $Wcm^(-2)$")
# plt.savefig('benchmark_Voesch_fig8b_10um_reflection05.png')